In [9]:
using DifferentialEquations
using Makie
using CairoMakie
using Colors, ColorSchemes
using StatsBase
using Random
using SpecialFunctions: erf
using LinearAlgebra
using Printf  # For simple number formatting

In [10]:
# Constants
const k = 2π  # three full eyes in the simulation domain
const ω = 4π  # two full oscillations in 1s
const g0 = 0.8  # wave amplitude
const t_on = 1.0  # amount of time before wave appears
const Δt_on = 0.07  # amount of time it takes wave to ramp up
const v_thermal = 0.8 * ω / k  # ensure a high gradient at the wave velocity

# Grid definitions
const x_grid = range(-1.5, 1.5, length=361)  # normalized spatial coordinates
const v_grid = range(-0.6 * ω / k, 2.1 * ω / k, length=201)  # velocity bounds

const num_samples = 100_000  # Reduced for testing
const frame_rate = 24
const duration = t_on + 2.0

# Custom colormap similar to the Python version
const colormap = cgrad(:thermal, rev=true)

function periodicize(x, minimum, maximum)
    return minimum .+ mod.(x .- minimum, maximum - minimum)
end

function format_velocity_label(v, ω, k)
    ratio = v / (ω / k)
    if isapprox(ratio, 0, atol=1e-3)
        return "0"
    elseif isapprox(ratio, 1, atol=1e-3)
        return "ω/k"
    elseif isapprox(ratio, -1, atol=1e-3)
        return "-ω/k"
    else
        # Use Printf for clean decimal formatting
        return @sprintf("%.2f ω/k", ratio)
    end
end

function derivative!(du, u, p, t)
    x = @view u[1:2:end]
    v = @view u[2:2:end]
    dxdt = @view du[1:2:end]
    dvdt = @view du[2:2:end]

    field_on = p[1]

    @. dxdt = v
    if field_on
        @. dvdt = g0 * (1 + erf((t - t_on) / Δt_on)) / 2 * sin(k * x - ω * t)
    else
        fill!(dvdt, 0.0)
    end
end

function generate_particles()
    Random.seed!(1)
    v0 = randn(num_samples) * v_thermal  # maxwellian initial distribution

    # Exclude particles off screen
    in_bounds = (v0 .> v_grid[1] - 0.1 * ω / k) .& (v0 .< v_grid[end] + 0.1 * ω / k)
    v0 = v0[in_bounds]
    x0 = rand(length(v0)) * (x_grid[end] - x_grid[1]) .+ x_grid[1]

    return x0, v0
end

function solve_equations(x0, v0, field_on=true)
    u0 = zeros(2 * length(x0))
    u0[1:2:end] .= x0
    u0[2:2:end] .= v0

    tspan = (0.0, duration)
    t = range(tspan..., step=1 / frame_rate)

    prob = ODEProblem(derivative!, u0, tspan, [field_on])
    sol = solve(prob, Tsit5(), saveat=t)

    # Extract positions and velocities
    x = sol[1:2:end, :]
    v = sol[2:2:end, :]

    return x, v, sol.t
end

solve_equations (generic function with 2 methods)

In [11]:
function create_phase_space_plot(x_grid, v_grid, t, x, v, field_on, wave_frame, trajectories, i)
    # Create new figure for each frame
    fig = Figure(size=(1200, 900))

    # Grid layout
    grid = fig[1, 1] = GridLayout()

    # Clear any existing content (important!)
    empty!(grid)

    # 1. Top row: Potential and Field
    ax_V = Axis(grid[1, 1], ylabel="Potential", ylabelsize=20, ylabelcolor=:orange)
    ax_E = Axis(grid[1, 1], ylabel="Field", ylabelsize=20, ylabelcolor=:purple,
        yaxisposition=:right)

    # 2. Right column: Velocity distribution
    ax_v = Axis(grid[2, 2], xlabel="Distribution", xlabelsize=20, xlabelcolor=RGB(0.38, 0.56, 0.62))

    # 3. Main plot: Phase space
    ax_image = Axis(grid[2, 1], xlabel="Position", ylabel="Velocity", ylabelsize=20)

    # Time display
    textbox = Label(grid[1, 2], @sprintf("t = %.1f s", t[i]),
        halign=:left, valign=:top, fontsize=20)

    # Calculate field and potential
    x_plot = wave_frame ? x_grid .+ ω / k * t[i] : x_grid
    E = field_on ? g0 * (1 + erf((t[i] - t_on) / Δt_on)) / 2 * sin.(k * x_plot .- ω * t[i]) : zeros(length(x_plot))
    ϕ = field_on ? g0 * (1 + erf((t[i] - t_on) / Δt_on)) / 2 * cos.(k * x_plot .- ω * t[i]) / k : zeros(length(x_plot))

    # Plot field and potential (clear old plots first)
    lines!(ax_E, x_plot, E, color=:purple, linewidth=1.4)
    lines!(ax_V, x_plot, ϕ, color=:orange, linewidth=1.4, linestyle=:dot)

    # Velocity distribution
    f_v = fit(Histogram, v[:, i], v_grid[1:2:end])
    barplot!(ax_v, f_v.weights ./ diff(f_v.edges[1]), f_v.edges[1][1:end-1],
        direction=:x, color=RGB(0.38, 0.56, 0.62))

    # Phase space density
    image = zeros(length(x_grid) - 1, length(v_grid) - 1)
    r_particle = (x_grid[2] - x_grid[1]) * 1.5
    for dx in range(-r_particle, r_particle, length=7),
        dy in range(-r_particle, r_particle, length=7)

        hypot(dx, dy) < r_particle * 1.1 || continue
        dv = dy / (x_grid[2] - x_grid[1]) * (v_grid[2] - v_grid[1])
        x_periodic = periodicize(x[:, i] .+ dx, x_grid[1], x_grid[end])
        h = fit(Histogram, (x_periodic, v[:, i] .+ dv), (x_grid, v_grid))
        image .+= h.weights
    end

    heatmap!(ax_image, x_grid[1:end-1], v_grid[1:end-1], image',
        colormap=colormap, colorrange=(0, maximum(image) * 0.75))

    # Set limits and ticks
    xlims!(ax_image, x_grid[1] + 0.21, x_grid[end] - 0.21)
    ax_image.xticks = -1.5:0.5:1.5

    if field_on
        ticks = range(v_grid[1], v_grid[end], step=0.5 * ω / k)
        ax_image.yticks = (ticks, [@sprintf("%.1f ω/k", t / (ω / k)) for t in ticks])
    else
        ax_image.yticks = range(v_grid[1], v_grid[end], step=1.0)
    end

    return fig
end

create_phase_space_plot (generic function with 1 method)

In [12]:
function generate_animation()
    println("Generating particles...")
    x0, v0 = generate_particles()

    println("Solving equations...")
    x, v, t = solve_equations(x0, v0)

    println("Creating animation frames...")
    frames = []

    # First pass to determine max histogram value for consistent scaling
    max_hist_value = 0.0
    sample_indices = round.(Int, range(1, length(t), length=min(10, length(t))))

    for i in sample_indices
        r_particle = (x_grid[2] - x_grid[1]) * 1.5
        conv_points = 7
        dx_values = range(-r_particle, r_particle, length=conv_points)
        dy_values = range(-r_particle, r_particle, length=conv_points)

        image = zeros(length(x_grid) - 1, length(v_grid) - 1)
        for dx in dx_values, dy in dy_values
            if hypot(dx, dy) < r_particle * 1.1
                dv = dy / (x_grid[2] - x_grid[1]) * (v_grid[2] - v_grid[1])
                x_periodic = periodicize(x[:, i] .+ dx, x_grid[1], x_grid[end])
                h = fit(Histogram, (x_periodic, v[:, i] .+ dv), (x_grid, v_grid))
                image .+= h.weights
            end
        end
        max_hist_value = max(max_hist_value, maximum(image))
    end

    vmax_value = max_hist_value * 0.75 * 1.5  # Using 1.5 for trajectories

    # Second pass to create frames
    for i in 1:length(t)
        if i % 10 == 0
            println("  Processing frame $i/$(length(t))...")
        end

        fig = create_phase_space_plot(x_grid, v_grid, t, x, v, true, true, true, i)

        # Save frame to memory
        push!(frames, fig)
    end

    println("Saving animation...")

    # Option 1: Save as GIF
    try
        save("landau_damping.gif", frames, fps=frame_rate)
        println("Successfully saved landau_damping.gif")
    catch e
        @warn "GIF saving failed, trying MP4" exception = e

        # Option 2: Correct MP4 saving approach
        try
            # Create a figure with the same size as your frames
            fig = Figure(size=(1200, 900))

            # Record animation properly
            record(fig, "landau_damping.mp4", 1:length(frames);
                framerate=frame_rate) do frame_num
                # Clear current content
                empty!(fig)

                # Recreate your plot for this frame
                # (You'll need to put your frame generation code here)
                # Example:
                ax = Axis(fig[1, 1])
                # ... recreate your specific plot for this frame ...

                # Or use your pre-generated frames if possible
                # This part needs to match your actual plotting code
            end
            println("Successfully saved landau_damping.mp4")
        catch e2
            @error "Both GIF and MP4 saving failed" exception = e2

            # Option 3: Save individual frames
            mkpath("frames")
            for (i, f) in enumerate(frames)
                save("frames/frame_$(lpad(i,4,'0')).png", f)
            end
            println("Saved individual frames in 'frames/' directory")
        end
    end
end

generate_animation (generic function with 1 method)

In [13]:
# Run the simulation
generate_animation()

Generating particles...
Solving equations...
Creating animation frames...


LoadError: MethodError: no method matching empty!(::GridLayout)
The function `empty!` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  empty!([91m::BitVector[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mbitarray.jl:1144[24m[39m
[0m  empty!([91m::Base.JuliaSyntax.ParseStream[39m)
[0m[90m   @[39m [90mBase[39m [90m/cache/build/tester-amdci5-12/julialang/julia-release-1-dot-11/base/JuliaSyntax/src/[39m[90m[4mparse_stream.jl:1132[24m[39m
[0m  empty!([91m::Base64.Buffer[39m)
[0m[90m   @[39m [36mBase64[39m [90m~/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/share/julia/stdlib/v1.11/Base64/src/[39m[90m[4mbuffer.jl:15[24m[39m
[0m  ...
